# Cleaning Data

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.metrics.pairwise import pairwise_kernels

In [2]:
# Read in CSV file
wine_file = 'db/final_data.csv'

In [3]:
df = pd.read_csv(wine_file)

In [4]:
df.head()

id country                                        description  \
0   3      US  Pineapple rind, lemon pith and orange blossom ...   
1   4      US  Much like the regular bottling from 2012, this...   
2   5   Spain  Blackberry and raspberry aromas show a typical...   
3   6   Italy  Here's a bright, informal red that opens with ...   
4   9  France  This has great depth of flavor with its fresh ...   

                          designation  points  price           province  \
0                Reserve Late Harvest      87     13           Michigan   
1  Vintner's Reserve Wild Child Block      87     65             Oregon   
2                        Ars In Vitro      87     15     Northern Spain   
3                             Belsito      87     16  Sicily & Sardinia   
4                         Les Natures      87     27             Alsace   

              region_1           region_2         taster_name  \
0  Lake Michigan Shore                NaN  Alexander Peartree   
1    Willamette Valley  Willamette Valley        Paul Gregutt   
2              Navarra                NaN   Michael Schachner   
3             Vittoria                NaN     Kerin O‚ÄôKeefe   
4               Alsace                NaN          Roger Voss   

  taster_twitter_handle                                              title  \
0                   NaN  St. Julian 2013 Reserve Late Harvest Riesling ...   
1          @paulgwine¬†  Sweet Cheeks 2012 Vintner's Reserve Wild Child...   
2           @wineschach  Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...   
3          @kerinokeefe   Terre di Giurfo 2013 Belsito Frappato (Vittoria)   
4            @vossroger  Jean-Baptiste Adam 2012 Les Natures Pinot Gris...   

              variety              winery  
0            Riesling          St. Julian  
1          Pinot Noir        Sweet Cheeks  
2  Tempranillo-Merlot              Tandem  
3            Frappato     Terre di Giurfo  
4          Pinot Gris  Jean-Baptiste Adam

In [5]:
# Drop rows without a price
df = df[df.price.notnull()]

In [6]:
df.count()

id                       40999
country                  40999
description              40999
designation              40999
points                   40999
price                    40999
province                 40999
region_1                 40999
region_2                 19752
taster_name              31458
taster_twitter_handle    30259
title                    40999
variety                  40999
winery                   40999
dtype: int64

In [7]:
# Drop rows without a designation
df = df[df.designation.notnull()]

In [8]:
df.count()

id                       40999
country                  40999
description              40999
designation              40999
points                   40999
price                    40999
province                 40999
region_1                 40999
region_2                 19752
taster_name              31458
taster_twitter_handle    30259
title                    40999
variety                  40999
winery                   40999
dtype: int64

In [9]:
df = df[df.region_1.notnull()]

In [10]:
df.count()

id                       40999
country                  40999
description              40999
designation              40999
points                   40999
price                    40999
province                 40999
region_1                 40999
region_2                 19752
taster_name              31458
taster_twitter_handle    30259
title                    40999
variety                  40999
winery                   40999
dtype: int64

In [11]:
len(df)

40999

In [12]:
# Export file as a CSV
df.to_csv('db/clean_final.csv')

# Recommender System

In [13]:
# Define the TD-IDF Vectorizer Object and remove english stop words
tfidf = TfidfVectorizer(stop_words='english')

In [14]:
# Construct the matrix
tfidf = tfidf.fit_transform(df['description'])

In [15]:
# Shape of matrix; 40,999 wines described by 20,595 different words
tfidf.shape

(40999, 20595)

In [16]:
# Compute the cosine similarity matrix
cosine_sim = pairwise_kernels(tfidf, tfidf, metric='cosine', n_jobs=-1)

In [17]:
# Construct a reverse map of indices and wine titles
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [18]:
# Function that takes wine title as an input and outputs most similar wines within our dataset
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get index of wine that matches title
    idx = indices[title]
    # Get the pairwise similarity scores of all wines with the input wine
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort wines based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 5 most similar wines
    sim_scores = sim_scores[1:6]
    # Get the wine indices
    wine_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar wines
    return df['title'].iloc[wine_indices]


In [20]:
get_recommendations("Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)")

13323    Kessler-Haak 2013 Four Pinot Noir (Sta. Rita H...
6057     Jacob Williams 2010 Eagle Ranch Vineyard Caber...
37635    Jacob Williams 2010 Eagle Ranch Vineyard Caber...
23240       Lava Cap 2005 Matagrano Sangiovese (El Dorado)
12949    Viv√°c Winery 2006 V. by Viv√°c Single Vineyar...
Name: title, dtype: object

In [21]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if taster exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [22]:
features = ['taster_name', 'variety', 'province']

for feature in features:
    df[feature] = df[feature].apply(clean_data)

In [23]:
def create_soup(x):
    return ' '.join(x['taster_name']) + ' ' + ' '.join(x['variety']) + ' '.join(x['province'])

In [24]:
# Create a new soup feature
df['soup'] = df.apply(create_soup, axis=1)

In [25]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [26]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [27]:
# Reset index of your main DataFrame and construct reverse mapping as before
df = df.reset_index()
indices = pd.Series(df.index, index=df['title'])

In [28]:
get_recommendations("Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)", cosine_sim2)

17            Erath 2010 Hyland Pinot Noir (McMinnville)
42     Z'IVO 2015 Ros√© of Pinot Noir (Eola-Amity Hills)
128    Silvan Ridge 2006 Reserve Pinot Noir (Willamet...
139    Amity 2006 Estate Single Vineyard Pinot Noir (...
142    Cherry Hill 2006 Papillon Estate Pinot Noir (W...
Name: title, dtype: object

In [29]:
result = get_recommendations("Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)", cosine_sim2)

In [30]:
for x in result.index:
    price = df.loc[x,'price']
    title = df.loc[x,'title']
    points = df.loc[x, 'points']
    variety = df.loc[x, 'variety']
    print(f"{title}, price: ${price}, points: {points}, grape type: {variety}")

Erath 2010 Hyland Pinot Noir (McMinnville), price: $50, points: 86, grape type: pinotnoir
Z'IVO 2015 Ros√© of Pinot Noir (Eola-Amity Hills), price: $25, points: 86, grape type: pinotnoir
Silvan Ridge 2006 Reserve Pinot Noir (Willamette Valley), price: $28, points: 85, grape type: pinotnoir
Amity 2006 Estate Single Vineyard Pinot Noir (Willamette Valley), price: $45, points: 85, grape type: pinotnoir
Cherry Hill 2006 Papillon Estate Pinot Noir (Willamette Valley), price: $22, points: 85, grape type: pinotnoir
